In [121]:
from bs4 import BeautifulSoup
import requests
import time, os

import math
import pandas as pd

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

chromedriver = "/Applications/chromedriver" # path to the chromedriver executable
os.environ["webdriver.chrome.driver"] = chromedriver


In [172]:
# load census data and drop the District of Columbia
county_census_info = pd.read_pickle("../Data/county_census_info.pkl")

county_census_info.drop('District of Columbia', level='STATE', inplace=True)

# county_census_info


In [209]:
# split the data frame into a list of dataframes per state

# type(county_census_info.index.unique(level='STATE'))

state_dataframes = []

for state in county_census_info.index.unique(level='STATE'):
    state_dataframes.append(county_census_info.loc[pd.IndexSlice[:,state],:])

    
# print(type(state_dataframes))
# print(type(state_dataframes[0]))
# print(len(state_dataframes))
# state_dataframes[0]

<class 'list'>
<class 'pandas.core.frame.DataFrame'>
50


,,"Population estimates, July 1, 2019, (V2019)","Population estimates base, April 1, 2010, (V2019)","Population, percent change - April 1, 2010 (estimates base) to July 1, 2019, (V2019)","Population, Census, April 1, 2010","Persons under 5 years, percent","Persons under 18 years, percent","Persons 65 years and over, percent","Female persons, percent","White alone, percent","Black or African American alone, percent(a)",...,"All firms, 2012","Men-owned firms, 2012","Women-owned firms, 2012","Minority-owned firms, 2012","Nonminority-owned firms, 2012","Veteran-owned firms, 2012","Nonveteran-owned firms, 2012","Population per square mile, 2010","Land area in square miles, 2010",FIPS Code
COUNTY,STATE,,,,,,,,,,,,,,,,,,,,,
Autauga County,Alabama,55869.0,54597.0,2.3,54571.0,5.9,23.2,16.0,51.5,76.3,20.1,...,2949.0,1499.0,1093.0,616.0,2160.0,285.0,2401.0,91.8,594.44,1001.0
Baldwin County,Alabama,223234.0,182265.0,22.5,182265.0,5.4,21.3,21.0,51.5,87.4,8.8,...,19732.0,10935.0,6505.0,2130.0,16893.0,2364.0,16183.0,114.6,1589.78,1003.0
Barbour County,Alabama,24686.0,27455.0,-10.1,27457.0,5.3,20.6,19.7,47.1,49.2,48.2,...,1687.0,755.0,760.0,657.0,941.0,147.0,1445.0,31.0,884.88,1005.0
Bibb County,Alabama,22394.0,22915.0,-2.3,22915.0,5.6,20.3,16.7,46.7,76.8,21.3,...,1310.0,958.0,254.0,205.0,1050.0,288.0,967.0,36.8,622.58,1007.0
Blount County,Alabama,57826.0,57322.0,0.9,57322.0,5.9,22.9,18.7,50.8,95.8,1.7,...,4550.0,2746.0,1400.0,368.0,4011.0,353.0,4030.0,88.9,644.78,1009.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tuscaloosa County,Alabama,209355.0,194667.0,7.5,194656.0,6.0,21.0,13.8,51.9,64.2,32.4,...,14255.0,7482.0,5171.0,3303.0,10334.0,1328.0,12014.0,147.3,1321.76,1125.0
Walker County,Alabama,63521.0,67020.0,-5.2,67023.0,6.1,22.2,19.5,51.2,90.9,6.2,...,4641.0,2597.0,1608.0,472.0,3997.0,367.0,4018.0,84.7,791.19,1127.0
Washington County,Alabama,16326.0,17580.0,-7.1,17581.0,5.5,22.1,19.7,51.0,66.0,23.3,...,1176.0,583.0,402.0,341.0,789.0,132.0,998.0,16.3,1080.21,1129.0


In [173]:
indices = county_census_info.index
print(indices[0])

('Autauga County', 'Alabama')


In [133]:
def isfloat(value_string_reduced):
  try:
    float(value_string_reduced)
    return float(value_string_reduced)
  except ValueError:
    return math.nan

def grabInitialMetric(soup, input_id):
    this_div = soup.find('div', id=input_id)
    this_span = this_div.find('span', id='initialMetric')
    return isfloat(this_span.text)
#     return isfloat(this_span.text.split(" ")[0].replace("<",''))

def grabPerMetric(soup, input_id):
    this_div = soup.find('div', id=input_id)
    sub_div = this_div.find('div', class_='rates')
    per_text = sub_div.text
    line_of_interest = per_text.split("\n")[1].split(" ")    
    while('' in line_of_interest): 
        line_of_interest.remove('')
    return isfloat(line_of_interest[0].replace("(",''))
    

In [180]:
def grabCovidData(soup):

    county_dict = {}

    svi_span = soup.find('span', id='svi_rank')
    svi_value = isfloat(svi_span.text)
    county_dict['SVI'] = svi_value
        
    ccvi_span = soup.find('span', id='ccvi_score')
    ccvi_value = isfloat(ccvi_span.text)
    county_dict['CCVI'] = ccvi_value

    county_dict['Cases'] = grabInitialMetric(soup, 'cases-timeseries-wrapper')
    county_dict['Cases per 100k'] = grabPerMetric(soup, 'cases-timeseries-wrapper')

    county_dict['Deaths'] = grabInitialMetric(soup, 'deaths-timeseries-wrapper')
    county_dict['Deaths per 100k'] = grabPerMetric(soup, 'deaths-timeseries-wrapper')

    county_dict['Percent Positivity'] = grabInitialMetric(soup, 'positivity-timeseries-wrapper')
    
    county_dict['Testing Volume'] = grabInitialMetric(soup, 'testing-timeseries-wrapper')
    county_dict['Testing Volume per 100k'] = grabPerMetric(soup, 'testing-timeseries-wrapper')

    county_dict['New Hospital Admissions'] = grabInitialMetric(soup, 'hospital-admissions-timeseries-wrapper')
    county_dict['New Hospital Admissions per 100 beds'] = grabPerMetric(soup, 'hospital-admissions-timeseries-wrapper')

    county_dict['Percent Beds Used (Covid)'] = grabInitialMetric(soup, 'hospital-percent-beds-timeseries-wrapper')
    
    county_dict['Percent ICU Beds Used (Covid)'] = grabInitialMetric(soup, 'hospital-percent-icu-beds-timeseries-wrapper')

    return county_dict

#     print(county_dict)

In [178]:
# state_input = 'Texas'
# county_input = 'Travis'

# state_input = 'Alaska'
# county_input = 'Petersburg'

test_index = 317
# test_index = 2000

state_input = indices[test_index][1]
county_input = indices[test_index][0].replace("County",'').replace("City and Borough",'').replace("Borough",'').replace("Census Area",'').rstrip()
print(state_input, county_input)
# county_input
state_input

Florida Alachua


'Florida'

In [216]:
def cleanCountyInput(county_string):
    return county_string.replace("County",'').replace("City and Borough",'').replace("Borough",'').replace("Census Area",'').rstrip()


In [220]:
# open the cdc website and choose the state and county from the drop down menu

cdc_county_website = "https://covid.cdc.gov/covid-data-tracker/#county-view"

driver = webdriver.Chrome(chromedriver)
driver.get(cdc_county_website)

time.sleep(10)

list_of_dicts = []
list_of_missed_rows = []

i = 0
j = 0
for state_frame in state_dataframes:
    i+=1
    if(i > 1): break
    
    for index, row in state_frame.iterrows():
        j+=1
#         if(j>1): break
            
        state_input = index[1]
        county_input = cleanCountyInput(index[0])

        print(state_input, county_input)

        select = Select(driver.find_element_by_id('list_select_state'))
        select.select_by_visible_text(state_input)

        time.sleep(1)

        select = Select(driver.find_element_by_id('list_select_county'))
        select.select_by_visible_text(county_input)

        time.sleep(15)

        # scrape data from the page

        soup = BeautifulSoup(driver.page_source)

        county_dict = grabCovidData(soup)
        county_dict['COUNTY'] = index[0]
        county_dict['STATE'] = index[1]        
        list_of_dicts.append(county_dict)
       
        
        
#     if(census_response.status_code == 200):
#         county_dict = grabCensusData(census_response)
#         county_dict['COUNTY'] = row['COUNTY NAME']
#         county_dict['STATE'] = row['STATE']
#         list_of_dicts.append(county_dict)
#     else:
#         print('Could not grab data for:',row['COUNTY NAME'], row['STATE'])
#         list_of_missed_rows.append((row['COUNTY NAME'], row['STATE']))
        


# close driver at end
driver.close()

# print(list_of_dicts)


Alabama Autauga
Alabama Baldwin
Alabama Barbour
Alabama Bibb
Alabama Blount
Alabama Bullock
Alabama Butler
Alabama Calhoun
Alabama Chambers
Alabama Cherokee
Alabama Chilton
Alabama Choctaw
Alabama Clarke
Alabama Clay
Alabama Cleburne
Alabama Coffee
Alabama Colbert
Alabama Conecuh
Alabama Coosa
Alabama Covington
Alabama Crenshaw
Alabama Cullman
Alabama Dale
Alabama Dallas
Alabama DeKalb
Alabama Elmore
Alabama Escambia
Alabama Etowah
Alabama Fayette
Alabama Franklin
Alabama Geneva
Alabama Greene
Alabama Hale
Alabama Henry
Alabama Houston
Alabama Jackson
Alabama Jefferson
Alabama Lamar
Alabama Lauderdale
Alabama Lawrence
Alabama Lee
Alabama Limestone
Alabama Lowndes
Alabama Macon
Alabama Madison
Alabama Marengo
Alabama Marion
Alabama Marshall
Alabama Mobile
Alabama Monroe
Alabama Montgomery
Alabama Morgan
Alabama Perry
Alabama Pickens
Alabama Pike
Alabama Randolph
Alabama Russell
Alabama St. Clair
Alabama Shelby
Alabama Sumter
Alabama Talladega
Alabama Tallapoosa
Alabama Tuscaloosa
Alabam

In [221]:
county_covid_info_df = pd.DataFrame(list_of_dicts)
county_covid_info_df.set_index(['COUNTY','STATE'],inplace=True)
county_covid_info_df

,,SVI,CCVI,Cases,Cases per 100k,Deaths,Deaths per 100k,Percent Positivity,Testing Volume,Testing Volume per 100k,New Hospital Admissions,New Hospital Admissions per 100 beds,Percent Beds Used (Covid),Percent ICU Beds Used (Covid)
COUNTY,STATE,,,,,,,,,,,,,
Autauga County,Alabama,0.44,0.80,257.0,460.00,NaN,8.95,17.02,1181.0,2113.87,36.0,54.55,NaN,NaN
Baldwin County,Alabama,0.22,0.38,894.0,400.48,NaN,1.79,18.41,3563.0,1596.08,78.0,20.58,NaN,NaN
Barbour County,Alabama,1.00,0.99,69.0,279.51,NaN,8.10,20.53,190.0,769.67,95.0,223.15,NaN,NaN
Bibb County,Alabama,0.60,0.80,108.0,482.27,NaN,8.93,20.04,509.0,2272.93,7.0,23.33,NaN,NaN
Blount County,Alabama,0.42,0.77,186.0,321.65,14.0,24.21,32.59,1025.0,1772.56,9.0,23.08,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Tuscaloosa County,Alabama,0.63,0.48,554.0,264.62,49.0,23.41,14.42,4007.0,1913.97,77.0,14.39,NaN,NaN
Walker County,Alabama,0.75,0.63,176.0,277.07,32.0,50.38,13.61,1433.0,2255.95,33.0,21.02,NaN,NaN
Washington County,Alabama,0.73,0.94,41.0,251.13,NaN,12.25,16.92,402.0,2462.33,3.0,11.11,NaN,NaN


In [223]:
county_covid_info_df.to_pickle("../Data/Covid/Alabama.pkl")